In [2]:
import pandas as pd
import numpy as np
import os
import turtle
filepath = "/Users/Jaron/Desktop/NCAA_Scouting/Shared_UMD_Baseball"

In [3]:
frames = []

for files in os.listdir(filepath):
    if files.endswith(".csv"):
        df = pd.read_csv(filepath + '/' + files)
            
        frames.append(df)
        
season = pd.concat(frames)

In [4]:
maryland_pitchers = season[(season["PitcherTeam"] == "Maryland")]

maryland_pitchers.head()

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,Unnamed: 0
0,1,4/30/22,2:03:24 PM,1.0,1.0,Jason Savacool,savacja42,NaN,Maryland,Ethan Odonnell,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,4/30/22,2:03:33 PM,1.0,2.0,Jason Savacool,savacja42,NaN,Maryland,Ethan Odonnell,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,4/30/22,2:03:43 PM,1.0,3.0,Jason Savacool,savacja42,NaN,Maryland,Ethan Odonnell,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4/30/22,2:04:05 PM,2.0,1.0,Jason Savacool,savacja42,NaN,Maryland,Anthony Calarco,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,4/30/22,2:04:15 PM,2.0,2.0,Jason Savacool,savacja42,NaN,Maryland,Anthony Calarco,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
maryland_pitchers['Catcher'].value_counts()

Luke Shliger       3898
Riley Langerman    1227
James Heffley       125
Name: Catcher, dtype: int64

In [6]:
called_pitches = season[(season["PitchCall"] == "StrikeCalled") 
                                   | (season["PitchCall"] == "BallCalled")]

called_pitches = called_pitches[["PitchCall", "PlateLocHeight", "PlateLocSide", "Catcher"]]

In [7]:
shliger = called_pitches[(called_pitches["Catcher"] == "Luke Shliger")]
langerman = called_pitches[(called_pitches["Catcher"] == "Riley Langerman")]
heffley = called_pitches[(called_pitches["Catcher"] == "James Heffley")]

shliger = shliger[["PitchCall", "PlateLocHeight", "PlateLocSide"]]
langerman = langerman[["PitchCall", "PlateLocHeight", "PlateLocSide"]]
heffley = heffley[["PitchCall", "PlateLocHeight", "PlateLocSide"]]

catcher_frames = {'Shliger':shliger, 'Langerman':langerman, 'Heffley':heffley}

In [8]:
#Catcher Viewpoint
#Dimensions in Rob's Umpire Dashboard
#UMD zone in 2022
edges = {"college":[-1.1, 1.1, 3.45, 1.55], 
         "rob":[-0.85, 0.85, 3.5, 1.5],
         "mlb":[-0.708333, 0.70833, 3.55, 1.71]}

shadow = 0.245

def get_zones(catcher_name, zone):
    right_edge = edges[zone][0]
    left_edge = edges[zone][1]
    top_edge = edges[zone][2]
    bottom_edge = edges[zone][3]
    catcher = catcher_frames[catcher_name]

    zone11 = catcher[((catcher["PlateLocSide"] >= (left_edge - shadow)) &
                             (catcher["PlateLocSide"] <= (left_edge + shadow))) &
                             ((catcher["PlateLocHeight"] <= (top_edge + shadow)) &
                             (catcher["PlateLocHeight"] >= (top_edge - shadow)))]

    zone12 = catcher[((catcher["PlateLocSide"] >= (right_edge + shadow)) &
                             (catcher["PlateLocSide"] <= (left_edge - shadow))) &
                             ((catcher["PlateLocHeight"] <= (top_edge + shadow)) &
                             (catcher["PlateLocHeight"] >= (top_edge - shadow)))]

    zone13 = catcher[((catcher["PlateLocSide"] >= (right_edge - shadow)) &
                             (catcher["PlateLocSide"] <= (right_edge + shadow))) &
                             ((catcher["PlateLocHeight"] <= (top_edge + shadow)) &
                             (catcher["PlateLocHeight"] >= (top_edge - shadow)))]

    zone14 = catcher[((catcher["PlateLocSide"] >= (left_edge - shadow)) &
                             (catcher["PlateLocSide"] <= (left_edge + shadow))) &
                             ((catcher["PlateLocHeight"] <= (top_edge - shadow)) &
                             (catcher["PlateLocHeight"] >= (bottom_edge + shadow)))]

    zone16 = catcher[((catcher["PlateLocSide"] >= (right_edge - shadow)) &
                             (catcher["PlateLocSide"] <= (right_edge + shadow))) &
                             ((catcher["PlateLocHeight"] <= (top_edge - shadow)) &
                             (catcher["PlateLocHeight"] >= (bottom_edge + shadow)))]

    zone17 = catcher[((catcher["PlateLocSide"] >= (left_edge - shadow)) &
                             (catcher["PlateLocSide"] <= (left_edge + shadow))) &
                             ((catcher["PlateLocHeight"] <= (bottom_edge + shadow)) &
                             (catcher["PlateLocHeight"] >= (bottom_edge - shadow)))]

    zone18 = catcher[((catcher["PlateLocSide"] >= (right_edge + shadow)) &
                             (catcher["PlateLocSide"] <= (left_edge - shadow))) &
                             ((catcher["PlateLocHeight"] <= (bottom_edge + shadow)) &
                             (catcher["PlateLocHeight"] >= (bottom_edge - shadow)))]

    zone19 = catcher[((catcher["PlateLocSide"] >= (right_edge - shadow)) &
                             (catcher["PlateLocSide"] <= (right_edge + shadow))) &
                             ((catcher["PlateLocHeight"] <= (bottom_edge + shadow)) &
                             (catcher["PlateLocHeight"] >= (bottom_edge - shadow)))]
    
    return [zone11, zone12, zone13, zone14, zone16, zone17, zone18, zone19]

In [9]:
catcher_names_list = ['Shliger', 'Langerman', 'Heffley']
catcher_zones = {}

for c in catcher_names_list: 
    zones = get_zones(c, "college")
    percentages = []
    for zone in zones: 
        try: 
            percentages.append(zone['PitchCall'].value_counts("StrikeCalled")["StrikeCalled"])
        except: 
            percentages.append(0)
    catcher_zones[c] = percentages
    
print(catcher_zones)
    

#zones[7]['PitchCall'].value_counts("StrikeCalled")

{'Shliger': [0.1, 0.24210526315789474, 0.12121212121212122, 0.4647887323943662, 0.6896551724137931, 0.21739130434782608, 0.47959183673469385, 0.46835443037974683], 'Langerman': [0.5714285714285714, 0.5428571428571428, 0.2727272727272727, 0.7272727272727273, 0.6, 0.058823529411764705, 0.3695652173913043, 0.0625], 'Heffley': [0.5, 0.3333333333333333, 0, 0.25, 1.0, 0, 0.75, 0]}
